In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import sklearn.cluster as sk_cluster
import sklearn.feature_extraction.text as sk_text
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [2]:
cleaned_data = pd.read_csv('cleaned_data.csv')
clinton_trump_texts = pd.read_table('clinton_trump_user_classes.txt', names=['UserID', 'Class'], sep='\t')

In [3]:
clinton_trump_texts_agg = cleaned_data.groupby('UserID')['Hash'].sum()
clinton_trump_sklearn = pd.DataFrame({'User_id':clinton_trump_texts_agg.index,'All_hashtag':clinton_trump_texts_agg.values})

In [28]:
lower_case = clinton_trump_sklearn.All_hashtag.str.lower()

In [21]:
vectorizer = sk_text.TfidfVectorizer()
matrix = vectorizer.fit_transform(clinton_trump_sklearn['All_hashtag'])

In [6]:
kmeans = sk_cluster.KMeans(n_clusters=2, n_init=10, max_iter=300)
kmeans_label = kmeans.fit_predict(matrix)

In [7]:
print(matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
print ('\n Cluster Centroids')
centroids = kmeans.cluster_centers_
print (centroids)


 Cluster Centroids
[[5.89108029e-04 1.03277298e-04 2.41903419e-04 ... 5.55801671e-06
  1.68866294e-05 1.31436412e-05]
 [1.13992567e-04 0.00000000e+00 3.04344550e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
